In [1]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [5]:
MathStudentsDF = pandas.read_csv("DataSets/student-mat.csv")
PorStudentsDF = pandas.read_csv("DataSets/student-por.csv")
MathStudentsDF
PorStudentsDF

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [29]:
MathStudentsReducedDF = MathStudentsDF.drop(['school', 'address', 'Mjob', 'Fjob', 'reason', 'traveltime', 'failures', 'activities', 'nursery', 'G1', 'G2', 'G3'], axis = 1)
MathStudentsReducedDF

,sex,age,famsize,Pstatus,Medu,Fedu,guardian,studytime,schoolsup,famsup,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,F,18,GT3,A,4,4,mother,2,yes,no,...,yes,no,no,4,3,4,1,1,3,6
1,F,17,GT3,T,1,1,father,2,no,yes,...,yes,yes,no,5,3,3,1,1,3,4
2,F,15,LE3,T,1,1,mother,2,yes,no,...,yes,yes,no,4,3,2,2,3,3,10
3,F,15,GT3,T,4,2,mother,3,no,yes,...,yes,yes,yes,3,2,2,1,1,5,2
4,F,16,GT3,T,3,3,father,2,no,yes,...,yes,no,no,4,3,2,1,2,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,M,20,LE3,A,2,2,other,2,no,yes,...,yes,no,no,5,5,4,4,5,4,11
391,M,17,LE3,T,3,1,mother,1,no,no,...,yes,yes,no,2,4,5,3,4,2,3
392,M,21,GT3,T,1,1,other,1,no,no,...,yes,no,no,5,5,3,3,3,3,3
393,M,18,LE3,T,3,2,mother,1,no,no,...,yes,yes,no,4,4,1,3,4,5,0


In [30]:
MathStudentsReducedDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sex        395 non-null    object
 1   age        395 non-null    int64 
 2   famsize    395 non-null    object
 3   Pstatus    395 non-null    object
 4   Medu       395 non-null    int64 
 5   Fedu       395 non-null    int64 
 6   guardian   395 non-null    object
 7   studytime  395 non-null    int64 
 8   schoolsup  395 non-null    object
 9   famsup     395 non-null    object
 10  paid       395 non-null    object
 11  higher     395 non-null    object
 12  internet   395 non-null    object
 13  romantic   395 non-null    object
 14  famrel     395 non-null    int64 
 15  freetime   395 non-null    int64 
 16  goout      395 non-null    int64 
 17  Dalc       395 non-null    int64 
 18  Walc       395 non-null    int64 
 19  health     395 non-null    int64 
 20  absences   395 non-null    int64

In [37]:
Input = MathStudentsReducedDF.iloc[:,14:17]
Input

,famrel,freetime,goout
0,4,3,4
1,5,3,3
2,4,3,2
3,3,2,2
4,4,3,2
...,...,...,...
390,5,5,4
391,2,4,5
392,5,5,3
393,4,4,1


In [38]:
OutputWorkday = MathStudentsReducedDF.iloc[:, 17]
OutputWorkday

0      1
1      1
2      2
3      1
4      1
      ..
390    4
391    3
392    3
393    3
394    3
Name: Dalc, Length: 395, dtype: int64

In [39]:
OutputWeekend = MathStudentsReducedDF.iloc[:, 18]
OutputWeekend

0      1
1      1
2      3
3      1
4      2
      ..
390    5
391    4
392    3
393    4
394    3
Name: Walc, Length: 395, dtype: int64

In [40]:
encoder = LabelEncoder()
encoder.fit(OutputWeekend)
encoded_output = encoder.transform(OutputWeekend)
dummy_y_weekend = np_utils.to_categorical(encoded_output)
dummy_y_weekend

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [59]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(9, input_dim=3, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [60]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5, verbose = 0)

In [61]:
kfold = KFold(n_splits=10, shuffle=True)

In [62]:
results = cross_val_score(estimator, Input, dummy_y_weekend, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 39.47% (6.35%)
